In [3]:
import pandas as pd
import mysql.connector

# Load your large dataset
df = pd.read_csv('names_data.csv')  # Replace with your actual DataFrame source

# Convert DataFrame to list of tuples
data = [tuple(row) for row in df.itertuples(index=False, name=None)]

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",      # replace with your MySQL username
    password="My$ql2024",  # replace with your MySQL password
    database="baby_names_db"
)
cursor = conn.cursor()

# Define your insert query
insert_query = """
    INSERT INTO names (State, Gender, Year, Name, Births)
    VALUES (%s, %s, %s, %s, %s)
"""

# Insert in chunks
batch_size = 10000
for i in range(0, len(data), batch_size):
    cursor.executemany(insert_query, data[i:i + batch_size])
    conn.commit()
    print(f"Inserted rows {i} to {i + batch_size}")

cursor.close()
conn.close()
print("Data upload complete.")

Inserted rows 0 to 10000
Inserted rows 10000 to 20000
Inserted rows 20000 to 30000
Inserted rows 30000 to 40000
Inserted rows 40000 to 50000
Inserted rows 50000 to 60000
Inserted rows 60000 to 70000
Inserted rows 70000 to 80000
Inserted rows 80000 to 90000
Inserted rows 90000 to 100000
Inserted rows 100000 to 110000
Inserted rows 110000 to 120000
Inserted rows 120000 to 130000
Inserted rows 130000 to 140000
Inserted rows 140000 to 150000
Inserted rows 150000 to 160000
Inserted rows 160000 to 170000
Inserted rows 170000 to 180000
Inserted rows 180000 to 190000
Inserted rows 190000 to 200000
Inserted rows 200000 to 210000
Inserted rows 210000 to 220000
Inserted rows 220000 to 230000
Inserted rows 230000 to 240000
Inserted rows 240000 to 250000
Inserted rows 250000 to 260000
Inserted rows 260000 to 270000
Inserted rows 270000 to 280000
Inserted rows 280000 to 290000
Inserted rows 290000 to 300000
Inserted rows 300000 to 310000
Inserted rows 310000 to 320000
Inserted rows 320000 to 330000
